In [ ]:
from pathlib import Path

from session_storage import SessionStorage

from convert_logs import convert_df

Заменим целевое действие.

Придумайте бинарный признак на основе колонки `total_view_time`

In [ ]:
def view_target(row):
    # здесь нужно задать условия для событий, которые будут приняты за "клики".
    # см. аналогичную функцию click_target в convert_logs.py
    
    # Ваш код
    
    raise NotImplementedError()

convert_df(
    path_from=Path("./data/VKVideoSessions.tsv"),
    path_to=Path("./data/VKVideoViewsSessions.txt"),
    target=view_target
)

Теперь необходимо загрузить кликовые и просмотровые сессии из .txt файлов, созданных по сессиям VK

In [ ]:
click_storage = SessionStorage("./data/VKVideoClickSessions.txt")
views_storage = None # Ваш код
click_test = click_storage.get_test_dataset()
views_test = views_storage.get_test_dataset()

Мы ещё не оценивали просмотровые модели по перплексии, давайте попробуем это сделать

In [ ]:
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity

from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.CTR import GCTR


def eval_model(model, storage):
    log_likelihood = LogLikelihood()
    perplexity = Perplexity()

    print(f"LL\t=\t{log_likelihood.evaluate(model, storage.get_test_sessions())}")
    print(f"PPL\t=\t{perplexity.evaluate(model, storage.get_test_sessions())[0]}")


# Ваш код
views_dbn_model = None 
views_ctr_model = None

In [ ]:
from catboost.metrics import QueryAUC

def apply_model(df, model):
    return df.apply(lambda x: model.predict_relevance(x.query, x.document), axis=1)

def eval_model(df, model):
    return QueryAUC().eval(
        df["clicked"].values,
        apply_model(df, model).values,
        group_id=df["query"].values
    )[0]

Обучим и оценим DBN модели на каждом датасете по AUC (модель должна оцениваться на соответствующем ей датасете, т.е. просмотровая -- на просмотровом, кликовая -- на кликовом)

In [ ]:
# Ваш код


**Замечание:** При формировании ground_truth метод для каждого датасета учитывается "клик". После того как вы заменили целевой признак на, например, просмотр >= 60 секунд -- он стал "кликом". Т.е. AUC для просмотровой модели, который мы тут видим, говорит нам о том, насколько хорошо модель располагает документы в порядке убывания факта просмотра.

Сами по себе Click AUC и Views AUC нельзя сравнивать между собой, однако мы пойдем на хитрость:

Мы хотим посмотреть, что будет, если мы применим обе DBN модели, обученные на кликах и просмотрах соответственно, только к просмотровому, т.е. более узкому датасету?

In [ ]:
# Ваш код


Как можно интерпретировать полученный результат?